In [1]:
import pandas as pd
import openml
import data_preprocess as dp

data_name = 'abalone'

data = openml.datasets.get_dataset(720)

X, y, categorical_indicator, attribute_names = data.get_data(
    target=data.default_target_attribute, dataset_format="dataframe"
)

c:\users\amr20\appdata\local\programs\python\python38\lib\site-packages\openml\datasets\functions.py:438: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  warnings.warn(


# Data Preprocessing

In [2]:
nominal = [b for a, b in zip(categorical_indicator, attribute_names) if a]
encoded_data = pd.get_dummies(X, columns=nominal)

columns = encoded_data.columns
updates = {}
for col in columns:
    if any(x in col for x in set((',', '[', ']', '<', '>'))):
        updates[col] = col.replace(',', '_').replace('[', '_').replace(']', '_').replace('<', 'less').replace('>', 'greater')
encoded_data = encoded_data.rename(columns=updates)

encoded_data.fillna(0, inplace=True)
encoded_data

,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Sex_M,Sex_F,Sex_I
0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.1500,1,0,0
1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700,1,0,0
2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,0,1,0
3,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550,1,0,0
4,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,0,0,1
...,...,...,...,...,...,...,...,...,...,...
4172,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,0,1,0
4173,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,1,0,0
4174,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,1,0,0
4175,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960,0,1,0


In [3]:
mapping = {'P': 1, 'N': 0}

y = y.replace(mapping)

In [4]:
import numpy as np
from sklearn.model_selection import train_test_split

train_data, left_out, train_label, y_left_out = train_test_split(encoded_data, y, test_size=0.4, random_state=42)
test_data, dev_data, test_label, dev_label = train_test_split(left_out, y_left_out, test_size=0.5, random_state=42)

print(f'train shape: {train_data.shape}\n test shape: {test_data.shape}\n dev shape: {dev_data.shape}')

train shape: (2506, 10)
 test shape: (835, 10)
 dev shape: (836, 10)


In [5]:
normalized_train_data = dp.min_max_normalize(train_data, train_data)

normalized_test_data = dp.min_max_normalize(test_data, train_data)

normalized_dev_data = dp.min_max_normalize(dev_data, train_data)

# Build graph

In [6]:
adj_matrix, index_to_name, name_to_index = dp.compute_adjacency_matrix(data=normalized_train_data, 
                                                                       self_loop_weight=20, threshold=0.2)
        

In [7]:
#oversampled_data, oversampled_labels = dp.oversample(normalized_train_data, train_label, oversampled_class=1, num_folds=2)

train = dp.transform_to_tensors(normalized_train_data, train_label, adj_matrix)

test = dp.transform_to_tensors(normalized_test_data, test_label, adj_matrix)

dev = dp.transform_to_tensors(normalized_dev_data, dev_label, adj_matrix)


In [8]:
from torch.utils.data import DataLoader
import torch

batch_size = 64

train_dataset = dp.Data(data=train)
val_dataset = dp.Data(data=dev)
test_dataset = dp.Data(data=test)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=False)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, drop_last=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=False)

# Train OGNNet

In [9]:
from sklearn.metrics import roc_auc_score, roc_curve, precision_score, recall_score, f1_score

In [10]:
import torch.nn as nn
from ognnet import train_model

gnn_model = train_model(input_dim=1,
            adj_matrix=adj_matrix, 
            train_dataloader=train_dataloader,
            val_dataloader=val_dataloader,
            data_name=data_name,
            num_classes=1,
            num_epochs=400,
            learning_rate=1e-04,
            normalize_adj=False)


 15%|████████████▍                                                                      | 6/40 [00:00<00:00, 55.43it/s]

Acc at dev is : 0.703
ROC is : 0.702,  prec 0.717696386730793,  recall 0.7020857659637796, f-score 0.6975256769374416
Acc at epoch : 1 is : 0.6755785942077637, loss : 29.101772636175156


 15%|████████████▍                                                                      | 6/40 [00:00<00:00, 52.95it/s]

Acc at dev is : 0.749
ROC is : 0.750,  prec 0.7686688311688312,  recall 0.7500858575451611, f-score 0.7446994154087776
Acc at epoch : 2 is : 0.7178770899772644, loss : 22.22740250825882


 15%|████████████▍                                                                      | 6/40 [00:00<00:00, 52.57it/s]

...

Acc at dev is : 0.794
ROC is : 0.7943769031755844,  prec 0.7944578037329669,  recall 0.7943769031755844, f-score 0.794253663003663
Acc at epoch : 398 is : 0.8056663870811462, loss : 16.81336146593094


  5%|████▏                                                                              | 2/40 [00:00<00:02, 13.84it/s]

Acc at dev is : 0.792
ROC is : 0.7921675023467729,  prec 0.7931070124262123,  recall 0.7921675023467729, f-score 0.7917468389224849
Acc at epoch : 399 is : 0.8088587522506714, loss : 16.462854266166687


100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:00<00:00, 55.31it/s]


Acc at dev is : 0.782
ROC is : 0.7826888293609909,  prec 0.784329303396661,  recall 0.7826888293609909, f-score 0.7820521635496883
Acc at epoch : 400 is : 0.8048683404922485, loss : 16.135922461748123


## Evaluate the trained model on the test set

In [11]:
import torch.nn as nn
from ognnet import OGNNet
from tqdm import tqdm



device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

input_dim = 1 
adj_matrix = torch.FloatTensor(adj_matrix)

gnn_model = OGNNet(input_dim, adj_matrix[0].shape[0], adj_matrix.to(device), 1).to(device)

torch.cuda.empty_cache()

gnn_model.load_state_dict(torch.load(f'{data_name}/ognnet-epoch[131]-0.807.model'))

print('*** Testing : ****')
gnn_model.eval()

correct = 0
count = 0
count_2 = 0

list_prediction = []
list_pob_pred = []
y_test = []

for i, data in tqdm(list(enumerate(test_dataloader))):
    inputs, labels = data

    inputs = inputs.to(device)
    labels = labels.to(device)

    outputs = gnn_model(inputs)
    list_pob_pred.extend(outputs.tolist())

    preds = (outputs.reshape(-1) > 0.5) * 1

    list_prediction.extend(preds.tolist())

    count+= torch.sum(preds == labels.data)

    for i in range(len(preds)):
        if preds[i] == labels[i] == 1:
            count_2 += 1

    y_test.extend(labels.tolist())

    torch.cuda.empty_cache()


roc = roc_auc_score(y_test, list_pob_pred)
prec = precision_score(y_test, list_prediction, average='macro')
recall = recall_score(y_test, list_prediction, average='macro')
f_score = f1_score(y_test, list_prediction, average='macro')

print('AUC is : {},  prec {},  recall {}, f-score {}' .format(roc, prec, recall, f_score))

 14%|███████████▊                                                                       | 2/14 [00:00<00:00, 18.77it/s]

*** Testing : ****


100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:00<00:00, 32.71it/s]

AUC is : 0.8825552373394063,  prec 0.8055996914326506,  recall 0.8052833465603901, f-score 0.8054228998849253
